In [11]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input
from sklearn.preprocessing import MinMaxScaler
from keras.utils import Sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,GlobalAveragePooling1D,LayerNormalization, MultiHeadAttention
from tensorflow.keras.optimizers import Adam

import warnings
warnings.filterwarnings('ignore')



data_clean = pd.read_csv("future_ss2312_tick.csv")

# 1. 数据加载和预处理
# 确保'close'列是数值型
data_clean['close'] = pd.to_numeric(data_clean['close'], errors='coerce')

price_features = ['昨收盘', '今开盘', '最高价', '最低价', '申买价一', '申卖价一']
for feature in price_features:
    data_clean[feature + '_diff'] = data_clean['close'] - data_clean[feature]

data_clean['trade_time'] = pd.to_datetime(data_clean['trade_time'])

# 将时间分解为小时和分钟
data_clean['hour'] = data_clean['trade_time'].dt.hour
data_clean['minute'] = data_clean['trade_time'].dt.minute

data_clean['close_diff'] = data_clean['close'].diff()

# Define label
data_clean['label'] = (data_clean['close'].shift(-100) > data_clean['close']).astype(int)

features = ['close_diff', '数量', 'hour', 'minute'] + [f + '_diff' for f in price_features]

# 3. 分割数据

# Now you can filter the data between two dates
train_data = data_clean[(data_clean['trade_time'] >= '2023-09-01 09:00:00') & 
                        (data_clean['trade_time'] < '2023-10-12 09:00:00')]

test_data = data_clean[(data_clean['trade_time'] >= '2023-10-12 09:00:00') & 
                        (data_clean['trade_time'] < '2023-10-20 09:00:00')]


# 初始化归一化器
scaler = MinMaxScaler(feature_range=(0, 1))

train_data[features] = scaler.fit_transform(train_data[features])


# 将 DataFrame 转换为 NumPy 数组
X_train = np.array(train_data[features])
y_train = np.array(train_data['label'])

# 删除 NaN 值
mask = ~np.isnan(X_train).any(axis=1)
X_train = X_train[mask]
y_train = y_train[mask]

# 首先，确保 X_train 和 X_test 没有 NaN 值
X_train = X_train[~np.isnan(X_train).any(axis=1)]
y_train = y_train[~np.isnan(X_train).any(axis=1)]

# Transformer Block
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [tf.keras.layers.Dense(ff_dim, activation="relu"), tf.keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TimeseriesGenerator(Sequence):
    def __init__(self, data, labels, length, stride=1, batch_size=32):
        self.data = data
        self.labels = labels
        self.length = length
        self.stride = stride
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil((len(self.data) - self.length) / float(self.stride * self.batch_size)))

    def __getitem__(self, idx):
        batch_x = []
        batch_y = []

        start = idx * self.batch_size * self.stride
        end = start + self.batch_size * self.stride + self.length

        for i in range(start, min(end, len(self.data) - self.length), self.stride):
            batch_x.append(self.data[i: i + self.length])
            batch_y.append(self.labels[i + self.length])

        return np.array(batch_x), np.array(batch_y)


# 定义时间步长
time_steps = 200  # 例如，选择200作为时间步长
stride = 1

# 创建数据生成器
train_generator = TimeseriesGenerator(X_train, y_train, length=time_steps, stride=stride, batch_size=32)

# 确定输入特征的维度
input_feature_dim = X_train.shape[1]

# 定义模型
embed_dim = input_feature_dim  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

model = Sequential()
model.add(Input(shape=(time_steps, input_feature_dim)))  # 明确指定输入形状
model.add(TransformerBlock(embed_dim, num_heads, ff_dim))
model.add(GlobalAveragePooling1D())  # 添加池化层以压缩时间维度
model.add(Dense(1, activation='sigmoid'))  # 输出层
model.compile(optimizer='adam', loss='binary_crossentropy')


# 使用生成器训练模型
model.fit(train_generator, epochs=5)

from keras.models import load_model
model.save('model_transformer.h5')



Epoch 1/5


2023-11-22 14:38:52.146424: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


19136/19136 [==============================] - 2279s 119ms/step - loss: 0.6222
Epoch 2/5
 7897/19136 [===========>..................] - ETA: 19:18 - loss: 0.6174

KeyboardInterrupt: 

In [9]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from keras.models import load_model

# 假设这些变量已经在上下文中定义
# model = load_model('gru_model.h5')
time_steps = 200
price_features = ['昨收盘', '今开盘', '最高价', '最低价', '申买价一', '申卖价一']
features = ['close_diff', '数量', 'hour', 'minute'] + [f + '_diff' for f in price_features]
scaler = MinMaxScaler(feature_range=(0, 1))

def prepare_data_for_prediction(test_data, time_steps, scaler):

    # 使用归一化
    scaled_data = scaler.fit_transform(test_data[features].dropna())

    # 重塑数据以适应模型
    X = np.array([scaled_data[i:i+time_steps] for i in range(len(scaled_data)-time_steps+1)])
    return X

# 准备数据
X_test = prepare_data_for_prediction(test_data, time_steps, scaler)
# 批量预测
predictions = model.predict(X_test, verbose=0)
# 确保test_data的索引与predictions对齐
aligned_test_data = test_data.iloc[time_steps - 1:]


initial_funds = 100000
funds = initial_funds
stock_quantity = 0
stock_price = 0
buy_threshold = 0.6
sold_threshold = 0.4
transactions = []
minute_count = 0
for prediction,  (index, row)  in zip(predictions, aligned_test_data.iterrows()):
    current_probability = prediction[0]
    if current_probability is not None:
        current_price = row['close']
        minute_count = minute_count + 1
            
        if current_probability > buy_threshold and stock_quantity == 0 :
            print(current_probability)
            stock_quantity = funds // current_price
            funds -= stock_quantity * current_price
            fee = stock_quantity * 2
            funds -= fee
            stock_price = current_price
            buy_price = current_price  # 记录买入价格
            print(f"Time: {row['trade_time']} - Action: BUY at {current_price}, Quantity: {stock_quantity},Funds:{funds}")
            transactions.append({
                'action': 'buy',
                'time': row['trade_time'],
                'price': current_price,
                'quantity': stock_quantity,
                'funds_remaining': funds
            })
            minute_count = 0
        elif minute_count > 200 and stock_quantity > 0 and current_probability<sold_threshold:
            print(current_probability)
            funds += stock_quantity * current_price
            price_diff = current_price - buy_price  # 计算价格差异
            # 判断价格差异是涨、跌还是平
            if price_diff > 0:
                direction = '涨'
            elif price_diff < 0:
                direction = '跌'
            else:
                direction = '平'
            print(f"Time: {row['trade_time']} - Action: SELL at {current_price}, Quantity: {stock_quantity},Funds:{funds}, Price Change: {direction}")
            transactions.append({
                'action': 'sell',
                'time': row['trade_time'],
                'price': current_price,
                'quantity': stock_quantity,
                'funds_remaining': funds,
                'price_change': direction
            })
            stock_quantity = 0
            buy_price = 0  # 重置买入价格为0

    
if len(transactions)>0:
    transactions_df = pd.DataFrame(transactions)
    print(transactions_df['price_change'].value_counts())
transactions_df.to_csv('transactions_tick_transformer.csv')

1.8407203
Time: 2023-10-17 09:05:39 - Action: BUY at 14805.0, Quantity: 6.0,Funds:11158.0
-14.871629
Time: 2023-10-17 09:07:56 - Action: SELL at 14830.0, Quantity: 6.0,Funds:100138.0, Price Change: 涨
0.65836716
Time: 2023-10-17 09:09:30 - Action: BUY at 14820.0, Quantity: 6.0,Funds:11206.0
-18.44158
Time: 2023-10-17 09:11:46 - Action: SELL at 14840.0, Quantity: 6.0,Funds:100246.0, Price Change: 涨
0.61604077
Time: 2023-10-17 09:15:19 - Action: BUY at 14820.0, Quantity: 6.0,Funds:11314.0
-11.009768
Time: 2023-10-17 09:18:59 - Action: SELL at 14825.0, Quantity: 6.0,Funds:100264.0, Price Change: 涨
0.73307836
Time: 2023-10-17 09:59:17 - Action: BUY at 14815.0, Quantity: 6.0,Funds:11362.0
-12.988117
Time: 2023-10-17 10:02:46 - Action: SELL at 14815.0, Quantity: 6.0,Funds:100252.0, Price Change: 平
2.3447926
Time: 2023-10-17 10:30:37 - Action: BUY at 14810.0, Quantity: 6.0,Funds:11380.0
-10.111759
Time: 2023-10-17 10:33:49 - Action: SELL at 14800.0, Quantity: 6.0,Funds:100180.0, Price Change: 